In [8]:
import numpy
import time
import pickle
from __future__ import print_function

In [9]:

def load_file(filepath):
    
    with open(filepath, 'r') as f:
        file = f.readlines()
    return file

src_file = load_file('data/file.en')
trg_file = load_file('data/file.de')
aligned_file = load_file('data/file.aligned')

In [43]:
def extract(trg_start, trg_end, src_start, src_end, alignments, trg_aligned, trg_len):    
  

    if trg_end < 0 :
        return [], []
    
    a = []
    
    for trg_word, src_word  in alignments:
        if trg_start <= trg_word <= trg_end:
            
            if (src_word < src_start or src_word > src_end):  
                return [], []
            else:
                a.append( (trg_word - trg_start, src_word - src_start) )
            
        
    E = []
    trg_s = trg_start
    
    while True:
        trg_e = trg_end
        while True:
            #E.append( (tuple(trg_words[trg_s:trg_e+1]) , tuple(src_words[src_start:src_end+1])) )
            E.append( (" ".join(trg_words[trg_s:trg_e+1]) , " ".join(src_words[src_start:src_end+1]) ))
            
            trg_e += 1
            
            if trg_e in trg_aligned or trg_e == len(trg_words):
                break 
    
        trg_s -= 1
        
        if trg_s in trg_aligned or trg_s < 0:
            break
    return E, a


def extract_phrases(src_words, trg_words, alignments, cutoff):
    
    
    if cutoff == -1:
        cutoff = len(src_words)
        
        
    trg_aligned = set()
    for (trg,_) in alignments:
        trg_aligned.add(trg)
    
    
    extracted_phrases = []
    extracted_alignments = []
    for src_start in range(len(src_words)):
        for src_end in range(src_start, min(src_start + cutoff, len(src_words))):

            trg_start = len(trg_words) - 1
            trg_end = - 1
            for  (trg,src) in alignments: 
                if src_start <= src <= src_end:
                    trg_start = min(trg, trg_start)
                    trg_end = max(trg, trg_end)
                    
            
            if(trg_end - trg_start > cutoff - 1):
                continue

            phrase_pairs, a = extract(trg_start, trg_end, src_start, src_end, alignments, trg_aligned, len(trg_words))
            
            if (phrase_pairs):
                extracted_phrases.extend(phrase_pairs)
                for i in phrase_pairs:
                    extracted_alignments.append(a);
            
    return extracted_phrases, extracted_alignments

In [44]:
# src_words = "michael assumes that he will stay in the house".split()
# trg_words = "michael geht davon aus , dass er im haus bleibt".split()
# alignments = [(0,0), (1,1), (2,1), (3,1), (5,2), (6,3), (9,4), (9,5), (7,6), (7,7), (8,8)]
 
# BP = extract_phrases(src_words, trg_words, alignments)
    
# freq_pairs = dict()
# freq_src = dict()
# freq_trg = dict()
    
# for pair in BP:
#     add_freq_dict(freq_pairs,pair)
#     add_freq_dict(freq_trg, pair[0])
#     add_freq_dict(freq_src, pair[1])

In [45]:
def add_freq_dict(dictionary, item):
    if item in dictionary:
        dictionary[item] += 1
    else:
        dictionary[item] = 1

        

freq_pairs = dict()
freq_src = dict()
freq_trg = dict()
    
start_time = time.time()
    
for i in range(len(src_file)):
    src_words = src_file[i].split()
    trg_words = trg_file[i].split()        
    alignments = [[int(a) for a in alignment.split('-')] for alignment in aligned_file[i].split()]
    
    phrase_pairs, extracted_alignments = extract_phrases(src_words, trg_words, alignments, 5)
    
#     for k in range(len(BP)):
#         print(BP[k], a[k])
#     break;
    
    for j, pair in enumerate(phrase_pairs):
        
        a = extracted_alignments[j]
        
        if pair in freq_pairs:
            freq_pairs[pair][0] += 1;
        else:
             freq_pairs[pair] = [1, a]
        
        
        add_freq_dict(freq_trg, pair[0])
        add_freq_dict(freq_src, pair[1])
        
    print(' \r%d / %d (%ds)'%(i+1,len(src_file), time.time() - start_time), end = '')

print()
print('Done!!!')
print('Total duration: %ds'%(time.time() - start_time))

print("#en phrases: %d"%(len(freq_src)))
print("#de phrases: %d"%(len(freq_trg)))
print("#(en,de) phrases: %d"%(len(freq_pairs)))

50000 / 50000 (75s)                                                                                                                                                                                                                                                                                                                                                                
Done!!!
Total duration: 75s
#en phrases: 1189230
#de phrases: 1386442
#(en,de) phrases: 2624229


In [48]:
len(phrase_pairs)

90

In [49]:
len(extracted_alignments)

90

In [53]:
for key in freq_pairs.keys()[10:20]:
    print(key, freq_pairs[key])

('darauf , wie die tiere', 'to the transfer of animals') [1, [(0, 0), (1, 0), (3, 1), (2, 2), (4, 4)]]
('geht', 'is needed') [1, [(0, 0)]]
('es ist nicht richtig , wenn hier', 'you are wrong to') [1, [(0, 0), (1, 1), (2, 2), (3, 2), (4, 3)]]
('auf die kommission an', "in the commission 's court") [1, [(0, 0), (1, 1), (2, 2), (2, 3), (3, 3), (3, 4)]]
('der ausgangspunkt', 'the point') [1, [(0, 0), (1, 1)]]
('die erarbeitung', 'communication') [1, [(0, 0)]]
('widersinnig und nicht', 'nonsensical and contrary to') [1, [(0, 0), (1, 1), (0, 2), (2, 2)]]
('informieren und dort eine kritik anzubringen', 'and then levy criticism') [1, [(0, 0), (1, 1), (2, 2), (4, 2), (3, 3)]]
('entwicklungspolitik steht im zusammenhang', 'development policies relates') [1, [(0, 0), (0, 1), (1, 2), (2, 2), (3, 2)]]
('f\xc3\xbcr die umsetzungsmodalit\xc3\xa4ten der', 'for the implementation methods for') [1, [(0, 0), (1, 1), (2, 2), (2, 3), (2, 4), (3, 4)]]


In [52]:
print('f\xc3\xbcr')

für


In [ ]:
# freq_pairs = dict()
# freq_src = dict()
# freq_trg = dict()
    
# start_time = time.time()
    
# for i in range(len(src_file)):
#     src_words = src_file[i].split()
#     trg_words = trg_file[i].split()        
#     alignments = [[int(a) for a in alignment.split('-')] for alignment in aligned_file[i].split()]
    
#     BP = extract_phrases(src_words, trg_words, alignments)
    
#     for pair in BP:
#         add_freq_dict(freq_pairs,pair)
#         add_freq_dict(freq_trg, pair[0])
#         add_freq_dict(freq_src, pair[1])
        
#     print(' \r%d / %d (%ds)'%(i+1,len(src_file), time.time() - start_time), end = '')

# print()
# print('Done!!!')
# print('Total duration: %ds'%(time.time() - start_time))

# print("#en phrases: %d"%(len(freq_src)))
# print("#de phrases: %d"%(len(freq_trg)))
# print("#(en,de) phrases: %d"%(len(freq_pairs)))

In [ ]:
#Save files
# with open("data/freq_src_small", 'wb') as file:
#     pickle.dump(freq_src, file)

# with open("data/freq_trg_small", 'wb') as file:
#     pickle.dump(freq_trg, file)

# with open("data/freq_pairs_small", 'wb') as file:
#     pickle.dump(freq_pairs, file)

with open("data/joined_freq_src", 'rb') as file:
    freq_src = pickle.load(file)

with open("data/joined_freq_trg", 'rb') as file:
    freq_trg = pickle.load(file)

with open("data/joined_freq_pairs", 'rb') as file:
    freq_pairs = pickle.load(file)
        
print("#en phrases: %d"%(len(freq_src)))
print("#de phrases: %d"%(len(freq_trg)))
print("#(en,de) phrases: %d"%(len(freq_pairs)))     

In [ ]:
# import operator
# sorted_freq_src = sorted(freq_src.items(), key=operator.itemgetter(1), reverse = True)
# sorted_freq_trg = sorted(freq_trg.items(), key=operator.itemgetter(1), reverse = True)
# sorted_freq_pairs = sorted(freq_pairs.items(), key=operator.itemgetter(1), reverse = True)

In [ ]:
src_total = 0
for _, freq in freq_src.iteritems():
    src_total += freq
    
trg_total = 0
for _, freq in freq_trg.iteritems():
    trg_total += freq
    
pairs_total = 0
for _, freq in freq_pairs.iteritems():
    pairs_total += freq

print("#en phrases: %d"%(src_total))
print("#de phrases: %d"%(trg_total))
print("#(en,de) phrases: %d"%(pairs_total))     

In [ ]:
for phrase_pair, freq in freq_pairs.iteritems():
    p_src_trg = 1.0 * freq / freq_trg[phrase_pair[0]]
    p_trg_src = 1.0 * freq / freq_src[phrase_pair[1]]
    
    freq_pairs[phrase_pair] = [freq, p_src_trg, p_trg_src]
    
    
    

In [ ]:
keys = freq_pairs.keys()[:5]

for key in keys:
    print(key[0], '|' , key[1], ':', freq_pairs[key])
    print(freq_trg[key[0]])
    print(freq_src[key[1]])


In [ ]:

with open("data/joined_freq_pairs_all", 'wb') as file:
    pickle.dump(freq_pairs, file)

In [23]:
a = 3
l = [3, 5]
print(a is list)
print(type(l) is list)


False
True


In [58]:

with open("data/freq_pairs_small", 'rb') as file:
    freq_pairs_small = pickle.load(file)

In [55]:
for key in old_freq_pairs.keys()[10:20]:
    print(key, old_freq_pairs[key])

('und verz\xc3\xb6gern', 'for prestige and rivalry among the various nations .') 1
('bei der ausarbeitung', 'responsible for preparing') 1
('( 1999 ) 410 - c5 - 0010 / 2000 - 2000 / 2004 ( cos )) , das europ\xc3\xa4ische parlament , den wirtschafts - und sozialausschu\xc3\x9f und den ausschu\xc3\x9f der regionen : transeurop\xc3\xa4ische netze - jahresbericht 1998 - gem\xc3\xa4\xc3\x9f artikel 16 der verordnung ( eg ) nr . 2236 / 95 \xc3\xbcber die grundregeln f\xc3\xbcr', '( 1999 ) 410 - c5 - 0010 / 2000 - 2000 / 2004 ( cos ) ] , the european parliament , the economic and social committee and the committee of the regions : trans - european networks 1998 annual report pursuant to article 16 of regulation ( ec ) no 2236 / 95 laying down general rules for the granting of community financial assistance in the field of trans -') 1
('ich bin \xc3\xbcberzeugt , da\xc3\x9f die irakische regierung bereit w\xc3\xa4re , die resolution 1284 zu akzeptieren', 'i am sure that the iraqi government is

In [56]:
len(old_freq_pairs)

8966987

In [57]:
len(freq_pairs)

2624229

In [59]:
len(freq_pairs_small)

2800861

In [60]:
for key in freq_pairs_small.keys()[10:20]:
    print(key, freq_pairs_small[key])

(('zu', 'zollen', ',', 'die', 'sich', 'f\xc3\xbcr', 'einen', 'fortbestand'), ('are', 'working', 'to', 'safeguard')) 1
(('schon',), ('already', 'been', 'raised')) 1
(('w\xc3\xa4hrend', 'der', 'portugiesischen', 'ratspr\xc3\xa4sidentschaft', 'vorangetrieben'), ('promoted', 'during', 'the', 'portuguese', 'presidency')) 1
(('vorbereitet', 'habe'), ('have', 'already')) 1
(('hunger', 'und', 'd\xc3\xbcrre', 'aufmerksam', 'machen'), ('starvation', 'and', 'drought')) 1
(('h\xc3\xa4tte', 'ich', 'vielleicht'), ('i', 'might')) 1
(('auch', 'selbst'), ('also', 'hold')) 1
(('tun', 'es', 'nicht'), ('do', 'not')) 1
(('diskussion',), ('discussion', 'may', 'make')) 1
(('aus', 'dieser', 'umweltkatastrophe', 'heraus'), ('this', 'ecological', 'disaster')) 1


In [61]:
freq_pairs[" ".join(['zu', 'zollen', ',', 'die', 'sich', 'f\xc3\xbcr', 'einen', 'fortbestand']), " ".join(('are', 'working', 'to', 'safeguard'))]

[1, [(0, 0), (1, 2), (2, 3), (3, 3)]]

In [63]:
freq_pairs[" ".join([ 'sich', 'f\xc3\xbcr', 'einen', 'fortbestand']), " ".join(('are', 'working', 'to', 'safeguard'))]

[1, [(0, 0), (1, 2), (2, 3), (3, 3)]]